In [150]:
import pandas as pd
import scipy, researchpy
from researchpy import ttest
import os, glob
import numpy as np
import json
import mpu.io
from glob2 import glob

In [202]:
gbm_seed = mpu.io.read('/home/polina/DeepLearningExamples/PyTorch/Segmentation/nnUNet/metrics/gbm_seed.json')
def ttest_pair(metric, df_1, df_2, name_1, name_2, correction = None):
    return ttest(df_1[metric], df_2[metric], group1_name = name_1,
                 group2_name= name_2, equal_variances=False, paired=True, correction = 8).iloc[4].values[1]

def make_df_fold(folder, dataset, exp, list_fold_0 =gbm_seed['fold_0'], list_fold_1=gbm_seed['fold_1'], list_fold_2=gbm_seed['fold_2']):
    """ for most of experiments name of experiment `exp` should include `_f`
    """
    _columns = ['Ids','Dice_1', 'Dice_2', 'Dice_3',
                'Sensitivity_1', 'Sensitivity_2', 'Sensitivity_3',
               'Specificity_1', 'Specificity_2', 'Specificity_3',
               'Surface_dice_1', 'Surface_dice_2', 'Surface_dice_3',
                'Hausdorff95_1', 'Hausdorff95_2', 'Hausdorff95_3',
               'Precision_1', 'Precision_2', 'Precision_3']
    metrics_all = pd.DataFrame(columns = _columns)
    print(exp)
    for i,sub in enumerate(os.listdir(folder)):
        if sub in list_fold_0:
            fold=0
            exp_f = f'{exp}_fold_{fold}'
#             print('{}/{}/*{}_epoch*_fold_{}_tta.json'.format(folder, sub, exp, fold))
            path_json = glob('{}/{}/{}_epoch_50*_fold_{}_tta.json'.format(folder, sub, exp, fold))
        elif sub in list_fold_1:
            fold=1
            exp_f = f'{exp}_fold_2'
            path_json = glob('{}/{}/{}_epoch_62*_fold_{}_tta.json'.format(folder, sub, exp, fold))
        elif sub in list_fold_2:
            fold=2
            exp_f = f'{exp}_fold_{fold}'
            path_json = glob('{}/{}/{}_epoch_78*_fold_{}_tta.json'.format(folder, sub, exp, fold))   
        print(path_json)    
        assert(len(path_json) != 0)
        metrics = mpu.io.read(path_json[0])
#         print(metrics)
        metrics = metrics[dataset][exp][f'fold={fold}']
        for each in metrics:
                    metrics_all.at[i,'Ids'] = sub
                    if each != 'Ids':
                        metrics_all.at[i,each] = np.round(metrics[each],3)

    return metrics_all

def make_df_fold_300(folder, dataset, exp, list_fold_0 =gbm_seed['fold_0'], list_fold_1=gbm_seed['fold_1'], list_fold_2=gbm_seed['fold_2']):
    """ for most of experiments name of experiment `exp` should include `_f`
    """
    _columns = ['Ids','Dice_1', 'Dice_2', 'Dice_3',
                'Sensitivity_1', 'Sensitivity_2', 'Sensitivity_3',
               'Specificity_1', 'Specificity_2', 'Specificity_3',
               'Surface_dice_1', 'Surface_dice_2', 'Surface_dice_3',
                'Hausdorff95_1', 'Hausdorff95_2', 'Hausdorff95_3',
               'Precision_1', 'Precision_2', 'Precision_3']
    metrics_all = pd.DataFrame(columns = _columns)
    print(exp)
    for i,sub in enumerate(os.listdir(folder)):
        if sub in list_fold_0:
            fold=0
            exp_f = f'{exp}_fold_{fold}'
#             print('{}/{}/{}_fold_0_unetr_epoch*_fold_{}_tta.json'.format(folder, sub, exp, fold))
            path_json = glob('{}/{}/{}_best_epoch*_fold_{}_tta.json'.format(folder, sub, exp, fold))
        elif sub in list_fold_1:
            fold=1
            exp_f=f'{exp}_fold_{fold}'
#             print('{}/{}/{}_epoch_65*_fold_{}_tta.json'.format(folder, sub, exp, fold))
            path_json = glob('{}/{}/{}_best_epoch*_fold_{}_tta.json'.format(folder, sub, exp, fold))
        elif sub in list_fold_2:
            fold=2
            exp_f=f'{exp}_fold_{fold}'
#             print('{}/{}/{}_epoch_57*_fold_{}_tta.json'.format(folder, sub, exp, fold))
            path_json = glob('{}/{}/{}_best_epoch*_fold_{}_tta.json'.format(folder, sub, exp, fold))   
        print(path_json)    
        assert(len(path_json) != 0)
        metrics = mpu.io.read(path_json[0])
#         print(metrics)
        metrics = metrics[dataset][exp][f'fold={fold}']
        for each in metrics:
                    metrics_all.at[i,'Ids'] = sub
                    if each != 'Ids':
                        metrics_all.at[i,each] = np.round(metrics[each],3)

    return metrics_all


def make_main_table(folder, dataset, fixed, moving, metrics, list_fold_0=gbm_seed['fold_0'], list_fold_1=gbm_seed['fold_1'], list_fold_2=gbm_seed['fold_2']):
    result_1 = pd.DataFrame()
    result_2 = pd.DataFrame()
    result_3 = pd.DataFrame()
    result = pd.DataFrame()
    df_1 = make_df_fold(folder,dataset, fixed, list_fold_0, list_fold_1, list_fold_2)
    
    for k, exp in enumerate(moving):
        df_2 = make_df_fold_300(folder,dataset, exp, list_fold_0, list_fold_1, list_fold_2)
#         for metric in [ 'Precision_1', 'Precision_2', 'Precision_3']:
#             print(np.round(df_2[metric].mean()))
#             print(np.round(df_2[metric].std()))
        for metric in metrics:
            print(metric)
            result = result.append({'Experiment': fixed,
                                     f'{metric}_Mean': np.round(df_1[metric].mean(),3),
                                     f'{metric}_STD': np.round(df_1[metric].std(),3)}, 
                                   ignore_index = True)
#             print(metric)
#             print(df_1[metric][0].dtype)
#             print(len(df_1[metric]))
#             print(df_1[metric].values)
#             print(np.isinf(df_1[metric]))
#             print(np.isinf(df_2[metric]))
            try:
                p_value = ttest_pair(metric, df_1, df_2, fixed, exp, correction = len(moving))
            except:
                continue
            if '1' in metric:
                result_1.at[k,'Experiment'] = exp
                result_1.at[k, f'{metric}_Mean'] = np.round(df_2[metric].mean(),3)
                result_1.at[k, f'{metric}_STD'] = np.round(df_2[metric].std(),3)
                result_1.at[k, f'{metric}_p_value'] = np.round(p_value,3)
                
            elif '2' in metric:
                result_2.at[k,'Experiment'] = exp
                result_2.at[k, f'{metric}_Mean'] = np.round(df_2[metric].mean(),3)
                result_2.at[k, f'{metric}_STD'] = np.round(df_2[metric].std(),3)
                result_2.at[k, f'{metric}_p_value'] = np.round(p_value,3)
            elif '3' in metric:
                result_3.at[k,'Experiment'] = exp
                result_3.at[k, f'{metric}_Mean'] = np.round(df_2[metric].mean(),3)
                result_3.at[k, f'{metric}_STD'] = np.round(df_2[metric].std(),3)
                result_3.at[k, f'{metric}_p_value'] = np.round(p_value,3)
#             result.at[k, f'{metric}_criterion'] = (p_value < 0.050)
    return result_1, result_2, result_3, result

In [193]:
! cd /results/gbm_infer && ls

6_histogram_fold_0  gbm_4a_resamp_100	       gbm_4b_n4_unetr_param
6_histogram_fold_1  gbm_4a_resamp_aug	       gbm_4d_susan
6_histogram_fold_2  gbm_4a_resamp_ct1	       gbm_4d_susan_unetr_param
gbm_1_reg_train     gbm_4a_resamp_flair        gbm_5_ss_shared
gbm_2a_interp	    gbm_4a_resamp_unetr_param  gbm_5_ss_shared_unetr_param
gbm_3a_atlas	    gbm_4a_resamp_wo_z
gbm_4a_resamp	    gbm_4b_n4


In [203]:
folder = '/results/metrics/gbm'
fixed = 'gbm_5_ss_shared'
gbm_seed = mpu.io.read('/home/polina/DeepLearningExamples/PyTorch/Segmentation/nnUNet/metrics/gbm_seed.json')
# moving = ['gbm_3a_atlas', 'gbm_5_ss_shared_unetr_param', gbm_4a_resamp_100','gbm_1_reg_train', 'gbm_4a_resamp_unetr_param', 'gbm_4a_resamp_aug', 'gbm_4a_resamp_wo_z', 'gbm_4b_n4', 'gbm_4b_n4_unetr_param', 'gbm_4d_susan', 'gbm_4d_susan_unetr_param']
moving = ['gbm_5_ss_shared_from_egd_TL']
dataset = 'gbm'

df_stat_1, df_stat_2, df_stat_3, result  = make_main_table(folder, dataset, fixed, moving,
                ['Dice_1', 'Dice_2', 'Dice_3',
                'Surface_dice_1', 'Surface_dice_2', 'Surface_dice_3',
                'Hausdorff95_1', 'Hausdorff95_2', 'Hausdorff95_3',
                'Sensitivity_1', 'Sensitivity_2', 'Sensitivity_3',
               'Specificity_1', 'Specificity_2', 'Specificity_3',
               'Precision_1',
#                 ]) 
                 'Precision_2', 
                 'Precision_3'
                ])

gbm_5_ss_shared
['/results/metrics/gbm/TCGA-02-0033/gbm_5_ss_shared_epoch_50-dice_mean_77_79_task_28_fold_0_tta.json']
['/results/metrics/gbm/TCGA-14-1825/gbm_5_ss_shared_epoch_50-dice_mean_77_79_task_28_fold_0_tta.json']
['/results/metrics/gbm/TCGA-06-0188/gbm_5_ss_shared_epoch_50-dice_mean_77_79_task_28_fold_0_tta.json']
['/results/metrics/gbm/TCGA-02-0046/gbm_5_ss_shared_epoch_78-dice_mean_77_36_task_28_fold_2_tta.json']
['/results/metrics/gbm/TCGA-12-1094/gbm_5_ss_shared_epoch_78-dice_mean_77_36_task_28_fold_2_tta.json']
['/results/metrics/gbm/TCGA-76-6663/gbm_5_ss_shared_epoch_78-dice_mean_77_36_task_28_fold_2_tta.json']
['/results/metrics/gbm/TCGA-06-0182/gbm_5_ss_shared_epoch_78-dice_mean_77_36_task_28_fold_2_tta.json']
['/results/metrics/gbm/TCGA-06-0164/gbm_5_ss_shared_epoch_78-dice_mean_77_36_task_28_fold_2_tta.json']
['/results/metrics/gbm/TCGA-02-0011/gbm_5_ss_shared_epoch_50-dice_mean_77_79_task_28_fold_0_tta.json']
['/results/metrics/gbm/TCGA-19-2631/gbm_5_ss_shared_epoch

In [204]:
df_stat_1.mean()

Dice_1_Mean               0.862
Dice_1_STD                0.099
Dice_1_p_value            0.740
Surface_dice_1_Mean       0.662
Surface_dice_1_STD        0.199
Surface_dice_1_p_value    0.203
Hausdorff95_1_Mean        8.858
Hausdorff95_1_STD         9.066
Hausdorff95_1_p_value     0.872
Sensitivity_1_Mean        0.811
Sensitivity_1_STD         0.126
Sensitivity_1_p_value     0.020
Specificity_1_Mean        0.999
Specificity_1_STD         0.001
Specificity_1_p_value     0.042
Precision_1_Mean          0.930
Precision_1_STD           0.088
Precision_1_p_value       0.001
dtype: float64

In [117]:
result.mean()

Dice_1_Mean             0.837
Dice_1_STD              0.127
Dice_2_Mean             0.789
Dice_2_STD              0.186
Dice_3_Mean             0.717
Dice_3_STD              0.207
Surface_dice_1_Mean     0.622
Surface_dice_1_STD      0.218
Surface_dice_2_Mean     0.604
Surface_dice_2_STD      0.231
Surface_dice_3_Mean     0.700
Surface_dice_3_STD      0.211
Hausdorff95_1_Mean     15.338
Hausdorff95_1_STD      20.437
Hausdorff95_2_Mean     10.715
Hausdorff95_2_STD      18.228
Hausdorff95_3_Mean      9.704
Hausdorff95_3_STD      18.808
Sensitivity_1_Mean      0.787
Sensitivity_1_STD       0.161
Sensitivity_2_Mean      0.752
Sensitivity_2_STD       0.197
Sensitivity_3_Mean      0.647
Sensitivity_3_STD       0.208
Specificity_1_Mean      0.999
Specificity_1_STD       0.001
Specificity_2_Mean      1.000
Specificity_2_STD       0.001
Specificity_3_Mean      1.000
Specificity_3_STD       0.001
Precision_1_Mean        0.917
Precision_1_STD         0.110
Precision_2_Mean        0.871
Precision_

In [200]:
! cd /results/metrics/gbm/TCGA-02-0033 && ls 

6_histogram_fold_0_epoch_249-dice_mean_73_16_task_31.0_fold_0_tta.json
6_histogram_fold_0_epoch_90-dice_mean_70_40_task_31.0_fold_0_tta.json
6_histogram_fold_0_from_bgpd_TL_best_epoch_87-dice_mean_76_78_task_31.0_fold_0_tta.json
6_histogram_fold_0_unetr_best_epoch_177-dice_mean_68_53_task_31.0_fold_0_tta.json
6_histogram_fold_0_unetr_epoch_177-dice_mean_68_53_task_31.0_fold_0_tta.json
6_histogram_fold_1_epoch_230-dice_mean_72_13_task_31.1_fold_1_tta.json
6_histogram_fold_1_from_bgpd_TL_best_epoch_89-dice_mean_76_44_task_31.1_fold_1_tta.json
6_histogram_fold_1_unetr_epoch_138-dice_mean_65_50_task_31.1_fold_1_tta.json
6_histogram_fold_2_epoch_243-dice_mean_74_42_task_31.2_fold_2_tta.json
6_histogram_fold_2_epoch_71-dice_mean_71_32_task_31.1_fold_1_tta.json
6_histogram_fold_2_epoch_73-dice_mean_72_44_task_31.2_fold_2_tta.json
6_histogram_fold_2_from_bgpd_TL_best_epoch_95-dice_mean_81_21_task_31.2_fold_2_tta.json
6_histogram_fold_2_unetr_epoch_194-dice_mean_71_36_task_31.2_fold_2_tta.json


In [176]:
! cd /results/metrics/gbm/TCGA-02-0033 && ls 50 62 78

ls: cannot access '50': No such file or directory
ls: cannot access '62': No such file or directory
ls: cannot access '78': No such file or directory


In [50]:
result.mean()

Dice_1_Mean            0.827
Dice_1_STD             0.128
Dice_2_Mean            0.827
Dice_2_STD             0.127
Dice_3_Mean            0.000
Dice_3_STD             0.000
Surface_dice_1_Mean    0.526
Surface_dice_1_STD     0.137
Surface_dice_2_Mean    0.525
Surface_dice_2_STD     0.136
Surface_dice_3_Mean    0.000
Surface_dice_3_STD     0.000
Hausdorff95_1_Mean       inf
Hausdorff95_1_STD        NaN
Hausdorff95_2_Mean       inf
Hausdorff95_2_STD        NaN
Hausdorff95_3_Mean       inf
Hausdorff95_3_STD        NaN
Sensitivity_1_Mean     0.830
Sensitivity_1_STD      0.153
Sensitivity_2_Mean     0.832
Sensitivity_2_STD      0.152
Sensitivity_3_Mean       NaN
Sensitivity_3_STD        NaN
Specificity_1_Mean     0.998
Specificity_1_STD      0.002
Specificity_2_Mean     0.998
Specificity_2_STD      0.002
Specificity_3_Mean     1.000
Specificity_3_STD      0.000
Precision_1_Mean       0.853
Precision_1_STD        0.116
Precision_2_Mean       0.851
Precision_2_STD        0.116
Precision_3_Me

In [313]:
print(df_stat_2.values)

[['gbm_4d_susan_unetr_param' 0.695 0.211 0.0 inf nan 0.0 0.82 0.217 0.0
  0.998 0.002 0.0 0.45 0.217 0.0]]


In [271]:
(0.866840+0.834330+0.867712)/3

0.8562940000000001

In [153]:
from tqdm import tqdm
import scipy
def make_pivot(exp_1 = 'metrics_gbm_1_reg_',
            exp_2 = 'metrics_gbm_1_reg_aug_',
            fold_isin = [0,1,2],
            columns = columns_1, result_df = result_df):
    
    df_1 = result_df[(result_df['exp'] == exp_1)&(result_df['fold'].isin(fold_isin))].sort_values(['Ids']).drop(['Unnamed: 0'], axis = 1)
    df_2 = result_df[(result_df['exp'] == exp_2)&(result_df['fold'].isin(fold_isin))].sort_values(['Ids']).drop(['Unnamed: 0'], axis = 1)

    df = pd.DataFrame()
    for metric in columns:
        table = pd.DataFrame()
        table = table.append({
                      f'{metric}_mean': np.round(df_1[metric][df_1[metric].notna()].mean(),3),
#                      f'{metric}_std':np.round(df_1[metric].std(),3),
                     f'{metric}_pvalue': np.round(scipy.stats.wilcoxon(df_1[metric], df_2[metric], correction= 8)[1],3)},
                    ignore_index = True)
        table = table.append({

                      f'{metric}_mean': np.round(df_2[metric][df_2[metric].notna()].mean(),3),
#                      f'{metric}_std': np.round(df_2[metric].std(),3),
                     f'{metric}_pvalue': np.round(scipy.stats.wilcoxon(df_1[metric], df_2[metric], correction= 8)[1],3)},
                    ignore_index = True)
        df = pd.concat([df,table],  axis =1)
    df['exp'] = ''
    df['exp'].loc[0] = exp_1
    df['exp'].loc[1] = exp_2

    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    return df[cols]

NameError: name 'columns_1' is not defined

In [160]:
!cd /results/metrics/gbm/TCGA-02-0006 && ls

6_histogram_fold_0_epoch_249-dice_mean_73_16_task_31.0_fold_0_tta.json
6_histogram_fold_0_epoch_90-dice_mean_70_40_task_31.0_fold_0_tta.json
6_histogram_fold_0_from_bgpd_TL_best_epoch_87-dice_mean_76_78_task_31.0_fold_0_tta.json
6_histogram_fold_0_unetr_best_epoch_177-dice_mean_68_53_task_31.0_fold_0_tta.json
6_histogram_fold_0_unetr_epoch_177-dice_mean_68_53_task_31.0_fold_0_tta.json
6_histogram_fold_1_epoch_230-dice_mean_72_13_task_31.1_fold_1_tta.json
6_histogram_fold_1_from_bgpd_TL_best_epoch_89-dice_mean_76_44_task_31.1_fold_1_tta.json
6_histogram_fold_1_unetr_epoch_138-dice_mean_65_50_task_31.1_fold_1_tta.json
6_histogram_fold_2_epoch_243-dice_mean_74_42_task_31.2_fold_2_tta.json
6_histogram_fold_2_epoch_71-dice_mean_71_32_task_31.1_fold_1_tta.json
6_histogram_fold_2_epoch_73-dice_mean_72_44_task_31.2_fold_2_tta.json
6_histogram_fold_2_from_bgpd_TL_best_epoch_95-dice_mean_81_21_task_31.2_fold_2_tta.json
6_histogram_fold_2_unetr_epoch_194-dice_mean_71_36_task_31.2_fold_2_tta.json


In [154]:
out_json = '/results/metrics/gbm/'
dataset = 'gbm'
task = 'gbm_4a_resamp'
fold = 'fold_0'
_columns = ['Ids','Dice_1', 'Dice_2', 'Dice_3',
                'Hausdorff95_1', 'Hausdorff95_2', 'Hausdorff95_3',
                'Sensitivity_1', 'Sensitivity_2', 'Sensitivity_3',
               'Specificity_1', 'Specificity_2', 'Specificity_3',
               'Surface_dice_1', 'Surface_dice_2', 'Surface_dice_3',
               'Precision_1', 'Precision_2', 'Precision_3']
metrics_all = pd.DataFrame(columns = _columns)
for i,sub in tqdm((enumerate(os.listdir(out_json)))):
    for sub_t in os.listdir(os.path.join(out_json, sub)):
        if f'{task}_epoch' in sub_t:
            if fold in sub_t:
                metrics = json.load(open(os.path.join(out_json, sub, sub_t)))
                metrics = metrics[dataset][task][fold.replace('_', '=')]
                for each in _columns:
                    metrics_all.at[i,'Ids'] = sub
                    if each != 'Ids':
                        metrics_all.at[i,each] = np.round(metrics[each],3)
#                 for each in metrics:
#                     metrics_all[each]
#                     metrics_all.at[i,'Ids'] = sub
#                     metrics_all.at[i,'Dice_1'] = metrics['Dice_1']   
#                     metrics_all.at[i,'Dice_2'] = metrics['Dice_2']
#                     metrics_all.at[i,'Dice_3'] = metrics['Dice_3']
#                     metrics_all.at[i,'Hausdorff95_1'] = metrics['Hausdorff95_1']
#                     metrics_all.at[i,'Hausdorff95_2'] = metrics['Hausdorff95_2']
#                     metrics_all.at[i,'Hausdorff95_3'] = metrics['Hausdorff95_3']
#                     metrics_all.at[i,'Sensitivity_1'] = metrics['Sensitivity_1']
#                     metrics_all.at[i,'Sensitivity_2'] = metrics['Sensitivity_2']
#                     metrics_all.at[i,'Sensitivity_3'] = metrics['Sensitivity_3']
#                     metrics_all.at[i,'Specificity_1'] = metrics['Specificity_1']
#                     metrics_all.at[i,'Specificity_2'] = metrics['Specificity_2']
#                     metrics_all.at[i,'Specificity_3'] = metrics['Specificity_3']
#                     metrics_all.at[i,'Surface_dice_1'] = metrics['Surface_dice_1']
#                     metrics_all.at[i,'Surface_dice_2'] = metrics['Surface_dice_2']
#                     metrics_all.at[i,'Surface_dice_3'] = metrics['Surface_dice_3']
#                     metrics_all.at[i,'Precision_1'] = metrics['Precision_1']
#                     metrics_all.at[i,'Precision_2'] = metrics['Precision_2']
#                     metrics_all.at[i,'Precision_3'] = metrics['Precision_3']
                    

102it [00:00, 193.66it/s]


In [159]:
metrics_all.sort_values('Dice_1', ascending=False)

,Ids,Dice_1,Dice_2,Dice_3,Hausdorff95_1,Hausdorff95_2,Hausdorff95_3,Sensitivity_1,Sensitivity_2,Sensitivity_3,Specificity_1,Specificity_2,Specificity_3,Surface_dice_1,Surface_dice_2,Surface_dice_3,Precision_1,Precision_2,Precision_3
40,TCGA-02-0085,0.968,0.924,0.9,0.781,6.5,3.906,0.959,0.883,0.865,1.0,1.0,1.0,0.97,0.883,0.914,0.978,0.969,0.94
54,TCGA-12-1098,0.962,0.908,0.905,1.953,1.953,2.0,0.961,0.841,0.83,1.0,1.0,1.0,0.929,0.858,0.858,0.963,0.987,0.996
68,TCGA-08-0512,0.957,0.948,0.923,1.811,1.5,1.5,0.935,0.93,0.877,1.0,1.0,1.0,0.692,0.729,0.718,0.981,0.966,0.975
75,TCGA-02-0106,0.956,0.935,0.89,1.562,2.471,2.344,0.927,0.882,0.815,1.0,1.0,1.0,0.935,0.857,0.904,0.987,0.996,0.981
26,TCGA-12-1598,0.955,0.885,0.825,2.009,6.289,4.581,0.954,0.817,0.717,0.999,1.0,1.0,0.923,0.771,0.787,0.956,0.966,0.972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,TCGA-19-5960,0.676,0.603,0.473,15.544,22.636,21.524,0.592,0.528,0.338,0.996,0.997,0.999,0.291,0.241,0.311,0.789,0.702,0.786
30,TCGA-08-0390,0.632,0.699,0.626,21.418,16.95,15.366,0.473,0.581,0.514,1.0,1.0,1.0,0.055,0.364,0.464,0.954,0.878,0.799
77,TCGA-19-5958,0.587,0.717,0.225,16.643,9.434,8.246,0.55,0.64,0.182,0.996,0.999,0.999,0.221,0.254,0.415,0.629,0.814,0.296
22,TCGA-06-0119,0.53,0.764,0.589,54.43,9.233,7.526,0.522,0.731,0.543,0.993,0.999,0.998,0.285,0.428,0.514,0.539,0.8,0.643


In [11]:
check_metrics = mpu.io.read('/results/metrics/gbm/TCGA-02-0006/gbm_1_reg_train_epoch_139-dice_mean_77_36_task_21_fold_2_tta.json')

In [12]:
check_metrics

{'gbm': {'gbm_1_reg_train': {'fold=2': {'Ids': 'TCGA-02-0006',
    'Dice_1': 0.07763368271451412,
    'Dice_2': 0.0,
    'Dice_3': 0.0,
    'Hausdorff95_1': 26.75206256651064,
    'Hausdorff95_2': inf,
    'Hausdorff95_3': inf,
    'Sensitivity_1': 0.04415925626515764,
    'Sensitivity_2': 0.0,
    'Sensitivity_3': 0.0,
    'Specificity_1': 0.9993822757894849,
    'Specificity_2': 1.0,
    'Specificity_3': 1.0,
    'Surface_dice_1': 0.2537246703185949,
    'Surface_dice_2': 0.0,
    'Surface_dice_3': 0.0,
    'Precision_1': 0.3208516886930984,
    'Precision_2': nan,
    'Precision_3': nan}}}}

In [29]:
!cd /results/metrics/gbm/EGD-0001 && ls -l

total 12
-rw-r--r-- 1 root root 624 Oct 13 22:47 EGD_train_from_brats_TL_epoch_21-dice_mean_84_64_task_14_fold_0_tta.json
-rw-r--r-- 1 root root 627 Oct 13 23:11 EGD_train_from_brats_TL_epoch_26-dice_mean_81_66_task_14_fold_1_tta.json
-rw-r--r-- 1 root root 627 Oct 13 23:36 EGD_train_from_brats_TL_epoch_69-dice_mean_82_74_task_14_fold_2_tta.json


In [166]:
p_value = ttest(24, 23, equal_variances=False, paired=True)

AttributeError: 'int' object has no attribute 'append'